In [ ]:
#| default_exp core

# <span style="color:DarkOrange"> Core </span>

> HierarchicalForecast contains pure Python implementations of hierarchical reconciliation methods as well as a `core.HierarchicalReconciliation` wrapper class that enables easy interaction with these methods through pandas DataFrames containing the hierarchical time series and the base predictions.<br><br> The `core.HierarchicalReconciliation` reconciliation class operates with the hierarchical time series pd.DataFrame `Y_df`, the base predictions pd.DataFrame `Y_hat_df`, the aggregation constraints matrix `S`. For more information on the creation of aggregation constraints matrix see the utils [aggregation method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br><br>

In [ ]:
#| export
import re
from inspect import signature, Parameter
from scipy.stats import norm
from typing import Callable, Dict, List, Optional

import numpy as np
import pandas as pd

from hierarchicalforecast.methods import _bootstrap_samples

In [ ]:
#| hide
from fastcore.test import test_close, test_eq, test_fail
from nbdev.showdoc import add_docs, show_doc

In [ ]:
#| exporti
def _build_fn_name(fn) -> str:
    fn_name = type(fn).__name__
    func_params = fn.__dict__
    args_to_remove = ['insample']
    if not func_params.get('nonnegative', False):
        args_to_remove += ['nonnegative']
    func_params = [f'{name}-{value}' for name, value in func_params.items() if name not in args_to_remove]
    if func_params:
        fn_name += '_' + '_'.join(func_params)
    return fn_name

In [ ]:
#| hide
# test fn name
from hierarchicalforecast.methods import BottomUp, MinTrace

In [ ]:
#| hide
test_eq(_build_fn_name(BottomUp()), 'BottomUp')
test_eq(
    _build_fn_name(MinTrace(method='ols')), 
    'MinTrace_method-ols'
)
test_eq(
    _build_fn_name(MinTrace(method='ols', nonnegative=True)), 
    'MinTrace_method-ols_nonnegative-True'
)

# <span style="color:DarkBlue"> core.HierarchicalReconciliation </span>

In [ ]:
#| export
class HierarchicalReconciliation:
    """Hierarchical Reconciliation Class.

    The `core.HierarchicalReconciliation` class allows you to efficiently fit multiple 
    HierarchicaForecast methods for a collection of time series and base predictions stored in 
    pandas DataFrames. The `Y_df` dataframe identifies series and datestamps with the unique_id and ds columns while the
    y column denotes the target time series variable. The `Y_h` dataframe stores the base predictions, 
    example ([AutoARIMA](https://nixtla.github.io/statsforecast/models.html#autoarima), [ETS](https://nixtla.github.io/statsforecast/models.html#autoets), etc.).

    **Parameters:**<br>
    `reconcilers`: A list of instantiated classes of the [reconciliation methods](https://nixtla.github.io/hierarchicalforecast/methods.html) module .<br>

    **References:**<br>
    [Rob J. Hyndman and George Athanasopoulos (2018). \"Forecasting principles and practice, Hierarchical and Grouped Series\".](https://otexts.com/fpp3/hierarchical.html)
    """
    def __init__(self,
                 reconcilers: List[Callable]):
        self.reconcilers = reconcilers
        self.insample = any([method.insample for method in reconcilers])

    def reconcile(self, 
                  Y_hat_df: pd.DataFrame,
                  S: pd.DataFrame,
                  tags: Dict[str, np.ndarray],
                  Y_df: Optional[pd.DataFrame] = None,
                  level: Optional[List[int]] = None,
                  bootstrap: bool = False):
        """Hierarchical Reconciliation Method.

        The `reconcile` method is analogous to SKLearn `fit` method, it applies different 
        reconciliation methods instantiated in the `reconcilers` list. 
        
        Most reconciliation methods can be described by the following convenient 
        linear algebra notation:

        $$\\tilde{\mathbf{y}}_{[a,b],\\tau} = \mathbf{S}_{[a,b][b]} \mathbf{P}_{[b][a,b]} \hat{\mathbf{y}}_{[a,b],\\tau}$$
        
        where $a, b$ represent the aggregate and bottom levels, $\mathbf{S}_{[a,b][b]}$ contains
        the hierarchical aggregation constraints, and $\mathbf{P}_{[b][a,b]}$ varies across 
        reconciliation methods. The reconciled predictions are $\\tilde{\mathbf{y}}_{[a,b],\\tau}$, and the 
        base predictions $\hat{\mathbf{y}}_{[a,b],\\tau}$.

        **Parameters:**<br>
        `Y_hat_df`: pd.DataFrame, base forecasts with columns `ds` and models to reconcile indexed by `unique_id`.<br>
        `Y_df`: pd.DataFrame, training set of base time series with columns `['ds', 'y']` indexed by `unique_id`.
        If a class of `self.reconciles` receives `y_hat_insample`, `Y_df` must include them as columns.<br>
        `S`: pd.DataFrame with summing matrix of size `(base, bottom)`, see [aggregate method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br>
        `tags`: Each key is a level and its value contains tags associated to that level.<br>
        `level`: float list 0-100, confidence levels for prediction intervals.<br>
        `bootstrap`: bool, whether or not to use bootstraped prediction intervals, alternative normality assumption.<br>

        **Returns:**<br>
        `y_tilde`: pd.DataFrame, with reconciled predictions.        
        """
        drop_cols = ['ds', 'y'] if 'y' in Y_hat_df.columns else ['ds']
        model_names = Y_hat_df.drop(columns=drop_cols, axis=1).columns.to_list()
        # store pi names
        pi_model_names = [name for name in model_names if ('-lo' in name or '-hi' in name)]
        #remove prediction intervals
        model_names = [name for name in model_names if name not in pi_model_names]
        uids = Y_hat_df.index.unique()
        # same order of Y_hat_df to prevent errors
        S_ = S.loc[uids]
        common_vals = dict(
            S=S_.values.astype(np.float32),
            idx_bottom=S_.index.get_indexer(S.columns),
            tags={key: S_.index.get_indexer(val) for key, val in tags.items()}
        )
        # we need insample values if 
        # we are using a method that requires them
        # or if we are performing boostrap
        if self.insample or bootstrap:
            if Y_df is None:
                raise Exception('you need to pass `Y_df`')
            common_vals['y_insample'] = Y_df.pivot(columns='ds', values='y').loc[uids].values.astype(np.float32)
        fcsts = Y_hat_df.copy()
        for reconcile_fn in self.reconcilers:
            reconcile_fn_name = _build_fn_name(reconcile_fn)
            has_fitted = 'y_hat_insample' in signature(reconcile_fn).parameters
            has_level = 'level' in signature(reconcile_fn).parameters
            for model_name in model_names:
                # should we calculate prediction intervals?
                pi_model_name = [pi_name for pi_name in pi_model_names if model_name in pi_name]
                pi = len(pi_model_name) > 0
                # Remember: pivot sorts uid
                y_hat_model = Y_hat_df.pivot(columns='ds', values=model_name).loc[uids].values
                if pi and has_level and level is not None and not bootstrap:
                    # we need to construct sigmah and add it
                    # to the common_vals
                    # to recover sigmah we only need 
                    # one prediction intervals
                    pi_col = pi_model_name[0]
                    sign = -1 if 'lo' in pi_col else 1
                    level_col = re.findall('[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', pi_col)
                    level_col = float(level_col[0])
                    z = norm.ppf(0.5 + level_col / 200)
                    sigmah = Y_hat_df.pivot(columns='ds', values=pi_col).loc[uids].values
                    sigmah = sign * (y_hat_model - sigmah) / z
                    common_vals['sigmah'] = sigmah
                    common_vals['level'] = level
                if (self.insample and has_fitted) or bootstrap:
                    if model_name in Y_df:
                        y_hat_insample = Y_df.pivot(columns='ds', values=model_name).loc[uids].values
                        y_hat_insample = y_hat_insample.astype(np.float32)
                        if has_fitted:
                            common_vals['y_hat_insample'] = y_hat_insample 
                        if bootstrap and has_level:
                            common_vals['bootstrap_samples'] = _bootstrap_samples(
                                y_insample=common_vals['y_insample'],
                                y_hat_insample=y_hat_insample, 
                                y_hat=y_hat_model, 
                                n_samples=1_000
                            )
                            common_vals['bootstrap'] = bootstrap
                            common_vals['level'] = level
                    else:
                        # some methods have the residuals argument
                        # but they don't need them
                        # ej MinTrace(method='ols')
                        common_vals['y_hat_insample'] = None
                kwargs = [key for key in signature(reconcile_fn).parameters if key in common_vals.keys()]
                kwargs = {key: common_vals[key] for key in kwargs}
                fcsts_model = reconcile_fn(y_hat=y_hat_model, **kwargs)
                fcsts[f'{model_name}/{reconcile_fn_name}'] = fcsts_model['mean'].flatten()
                if (pi and has_level and level is not None) or (bootstrap and level is not None):
                    for lv in level:
                        fcsts[f'{model_name}/{reconcile_fn_name}-lo-{lv}'] = fcsts_model[f'lo-{lv}'].flatten()
                        fcsts[f'{model_name}/{reconcile_fn_name}-hi-{lv}'] = fcsts_model[f'hi-{lv}'].flatten()
                    del common_vals['level']
                    if not bootstrap:
                        del common_vals['sigmah']
                    else:
                        del common_vals['bootstrap_samples']
                        del common_vals['bootstrap']
                if self.insample and has_fitted:
                    del common_vals['y_hat_insample']
        return fcsts

In [ ]:
show_doc(HierarchicalReconciliation, 
         name='init', title_level=3)

---

### init

>      init (reconcilers:List[Callable])

Hierarchical Reconciliation Class.

The `core.HierarchicalReconciliation` class allows you to efficiently fit multiple 
HierarchicaForecast methods for a collection of time series and base predictions stored in 
pandas DataFrames. The `Y_df` dataframe identifies series and datestamps with the unique_id and ds columns while the
y column denotes the target time series variable. The `Y_h` dataframe stores the base predictions, 
example ([AutoARIMA](https://nixtla.github.io/statsforecast/models.html#autoarima), [ETS](https://nixtla.github.io/statsforecast/models.html#autoets), etc.).

**Parameters:**<br>
`reconcilers`: A list of instantiated classes of the [reconciliation methods](https://nixtla.github.io/hierarchicalforecast/methods.html) module .<br>

**References:**<br>
[Rob J. Hyndman and George Athanasopoulos (2018). "Forecasting principles and practice, Hierarchical and Grouped Series".](https://otexts.com/fpp3/hierarchical.html)

In [ ]:
show_doc(HierarchicalReconciliation)

---

### HierarchicalReconciliation

>      HierarchicalReconciliation (reconcilers:List[Callable])

Hierarchical Reconciliation Class.

The `core.HierarchicalReconciliation` class allows you to efficiently fit multiple 
HierarchicaForecast methods for a collection of time series and base predictions stored in 
pandas DataFrames. The `Y_df` dataframe identifies series and datestamps with the unique_id and ds columns while the
y column denotes the target time series variable. The `Y_h` dataframe stores the base predictions, 
example ([AutoARIMA](https://nixtla.github.io/statsforecast/models.html#autoarima), [ETS](https://nixtla.github.io/statsforecast/models.html#autoets), etc.).

**Parameters:**<br>
`reconcilers`: A list of instantiated classes of the [reconciliation methods](https://nixtla.github.io/hierarchicalforecast/methods.html) module .<br>

**References:**<br>
[Rob J. Hyndman and George Athanasopoulos (2018). "Forecasting principles and practice, Hierarchical and Grouped Series".](https://otexts.com/fpp3/hierarchical.html)

In [ ]:
show_doc(HierarchicalReconciliation.reconcile,
         name='reconcile', title_level=3)

---

### reconcile

>      reconcile (Y_hat_df:pandas.core.frame.DataFrame,
>                 S:pandas.core.frame.DataFrame, tags:Dict[str,numpy.ndarray],
>                 Y_df:Optional[pandas.core.frame.DataFrame]=None,
>                 level:Optional[List[int]]=None, bootstrap:bool=False)

Hierarchical Reconciliation Method.

The `reconcile` method is analogous to SKLearn `fit` method, it applies different 
reconciliation methods instantiated in the `reconcilers` list. 

Most reconciliation methods can be described by the following convenient 
linear algebra notation:

$$\tilde{\mathbf{y}}_{[a,b],\tau} = \mathbf{S}_{[a,b][b]} \mathbf{P}_{[b][a,b]} \hat{\mathbf{y}}_{[a,b],\tau}$$

where $a, b$ represent the aggregate and bottom levels, $\mathbf{S}_{[a,b][b]}$ contains
the hierarchical aggregation constraints, and $\mathbf{P}_{[b][a,b]}$ varies across 
reconciliation methods. The reconciled predictions are $\tilde{\mathbf{y}}_{[a,b],\tau}$, and the 
base predictions $\hat{\mathbf{y}}_{[a,b],\tau}$.

**Parameters:**<br>
`Y_hat_df`: pd.DataFrame, base forecasts with columns `ds` and models to reconcile indexed by `unique_id`.<br>
`Y_df`: pd.DataFrame, training set of base time series with columns `['ds', 'y']` indexed by `unique_id`.
If a class of `self.reconciles` receives `y_hat_insample`, `Y_df` must include them as columns.<br>
`S`: pd.DataFrame with summing matrix of size `(base, bottom)`, see [aggregate method](https://nixtla.github.io/hierarchicalforecast/utils.html#aggregate).<br>
`tags`: Each key is a level and its value contains tags associated to that level.<br>
`level`: float list 0-100, confidence levels for prediction intervals.<br>
`bootstrap`: bool, whether or not to use bootstraped prediction intervals, alternative normality assumption.<br>

**Returns:**<br>
`y_tilde`: pd.DataFrame, with reconciled predictions.

In [ ]:
#| hide
from hierarchicalforecast.methods import (
    BottomUp, TopDown, MiddleOut, MinTrace, ERM,
)
from hierarchicalforecast.utils import aggregate

In [ ]:
#| hide
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')

# non strictly hierarchical structure
hiers_grouped = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]
# strictly hierarchical structure
hiers_strictly = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'State', 'Region'], 
]

# getting df
hier_grouped_df, S_grouped, tags_grouped = aggregate(df, hiers_grouped)
hier_strict_df, S_strict, tags_strict = aggregate(df, hiers_strictly)

In [ ]:
#| hide
hier_grouped_df['y_model'] = hier_grouped_df['y']
# we should be able to recover y using the methods
hier_grouped_df_h = hier_grouped_df.groupby('unique_id').tail(12)
ds_h = hier_grouped_df_h['ds'].unique()
hier_grouped_df = hier_grouped_df.query('~(ds in @ds_h)')
#adding noise to `y_model` to avoid perfect fited values
hier_grouped_df['y_model'] += np.random.uniform(-1, 1, len(hier_grouped_df))

#hierachical reconciliation
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='wls_var'),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols', nonnegative=True),
    MinTrace(method='wls_struct', nonnegative=True),
    MinTrace(method='wls_var', nonnegative=True),
    MinTrace(method='mint_shrink', nonnegative=True),
    # ERM recovers but needs bigger eps
    #ERM(method='reg_bu', lambda_reg=None),
])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_df_h, Y_df=hier_grouped_df, 
                            S=S_grouped, tags=tags_grouped)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    if 'ERM' in model:
        eps = 3
    else:
        eps = 1e-5
    test_close(reconciled['y'], reconciled[model], eps=eps)

/Users/fedex/projects/hierarchicalforecast/hierarchicalforecast/methods.py:467: RuntimeWarning: invalid value encountered in sqrt
  W_sqrt = np.sqrt(W_inv)


ValueError: array must not contain infs or NaNs

In [ ]:
#| hide
# test reconcile method without insample
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
])
reconciled = hrec.reconcile(Y_hat_df=hier_grouped_df_h,
                            S=S_grouped, tags=tags_grouped)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    if 'ERM' in model:
        eps = 3
    else:
        eps = 1e-5
    test_close(reconciled['y'], reconciled[model], eps=eps)

In [ ]:
#| hide
# top down should break
# with non strictly hierarchical structures
hrec = HierarchicalReconciliation([TopDown(method='average_proportions')])
test_fail(
    hrec.reconcile,
    contains='requires strictly hierarchical structures',
    args=(hier_grouped_df_h, S_grouped, tags_grouped,  hier_grouped_df,)
)

In [ ]:
#| hide
# methods should work with
# srtictly hierarchical structures
#| hide
hier_strict_df['y_model'] = hier_strict_df['y']
# we should be able to recover y using the methods
hier_strict_df_h = hier_strict_df.groupby('unique_id').tail(12)
ds_h = hier_strict_df_h['ds'].unique()
hier_strict_df = hier_strict_df.query('~(ds in @ds_h)')
#adding noise to `y_model` to avoid perfect fited values
hier_strict_df['y_model'] += np.random.uniform(-1, 1, len(hier_strict_df))

middle_out_level = 'Country/State'
# hierarchical reconciliation
hrec = HierarchicalReconciliation(reconcilers=[
    #these methods should reconstruct the original y
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct'),
    MinTrace(method='wls_var'),
    MinTrace(method='mint_shrink'),
    # top down doesnt recover the original y
    # but it should recover the total level
    TopDown(method='forecast_proportions'),
    TopDown(method='average_proportions'),
    TopDown(method='proportion_averages'),
    # middle out doesnt recover the original y
    # but it should recover the total level
    MiddleOut(middle_level=middle_out_level, top_down_method='forecast_proportions'),
    MiddleOut(middle_level=middle_out_level, top_down_method='average_proportions'),
    MiddleOut(middle_level=middle_out_level, top_down_method='proportion_averages'),
    # ERM recovers but needs bigger eps
    #ERM(method='reg_bu', lambda_reg=None),
])
reconciled = hrec.reconcile(
    Y_hat_df=hier_strict_df_h, 
    Y_df=hier_strict_df, 
    S=S_strict, 
    tags=tags_strict
)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    if 'ERM' in model:
        eps = 3
    else:
        eps = 1e-5
    if 'TopDown' in model:
        if 'forecast_proportions' in model:
            test_close(reconciled['y'], reconciled[model], eps)
        else:
            # top down doesnt recover the original y
            test_fail(
                test_close,
                args=(reconciled['y'], reconciled[model], eps),
            )
        # but it should recover the total level
        total_tag = tags_strict['Country']
        test_close(reconciled['y'].loc[total_tag], 
                   reconciled[model].loc[total_tag], 1e-2)
    elif 'MiddleOut' in model:
        if 'forecast_proportions' in model:
            test_close(reconciled['y'], reconciled[model], eps)
        else:
            # top down doesnt recover the original y
            test_fail(
                test_close,
                args=(reconciled['y'], reconciled[model], eps),
            )
        # but it should recover the total level
        total_tag = tags_strict[middle_out_level]
        test_close(reconciled['y'].loc[total_tag], 
                   reconciled[model].loc[total_tag], 1e-2)
    else:
        test_close(reconciled['y'], reconciled[model], eps)

In [ ]:
#| hide
#test methods that dont use residuals
#even if their signature includes
#that argument
hrec = HierarchicalReconciliation([MinTrace(method='ols')])
reconciled = hrec.reconcile(
    Y_hat_df=hier_grouped_df_h, 
    Y_df=hier_grouped_df.drop(columns=['y_model']), 
    S=S_grouped, 
    tags=tags_grouped
)
for model in reconciled.drop(columns=['ds', 'y']).columns:
    test_close(reconciled['y'], reconciled[model])

In [ ]:
#| hide
reconciled.loc[tags_grouped['Country/State']]

In [ ]:
#| hide
#test methods bootrap prediction
#intervals
hrec = HierarchicalReconciliation([BottomUp()])
reconciled = hrec.reconcile(hier_grouped_df_h, 
                            Y_df=hier_grouped_df, S=S_grouped, tags=tags_grouped,
                            level=[80, 90], bootstrap=True)
total = reconciled.loc[tags_grouped['Country/State/Region/Purpose']].groupby('ds').sum().reset_index()
pd.testing.assert_frame_equal(
    total[['ds', 'y_model/BottomUp']],
    reconciled.loc['Australia'][['ds', 'y_model/BottomUp']].reset_index(drop=True)
)

# <span style="color:DarkBlue"> Example </span>

In [ ]:
#| eval: false
import numpy as np
import pandas as pd

from statsforecast.core import StatsForecast
from statsforecast.models import ETS, Naive

from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, MinTrace

# Load TourismSmall dataset
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')

# Create hierarchical seires based on geographic levels and purpose
# And Convert quarterly ds string to pd.datetime format
hierarchy_levels = [['Country'],
                    ['Country', 'State'], 
                    ['Country', 'Purpose'], 
                    ['Country', 'State', 'Region'], 
                    ['Country', 'State', 'Purpose'], 
                    ['Country', 'State', 'Region', 'Purpose']]

Y_df, S, tags = aggregate(df=df, spec=hierarchy_levels)
qs = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(qs, freq='Q').to_timestamp()
Y_df = Y_df.reset_index()

# Split train/test sets
Y_test_df  = Y_df.groupby('unique_id').tail(4)
Y_train_df = Y_df.drop(Y_test_df.index)

# Compute base auto-ETS predictions
# Careful identifying correct data freq, this data quarterly 'Q'
fcst = StatsForecast(df=Y_train_df,
                     #models=[ETS(season_length=12), Naive()],
                     models=[Naive()],
                     freq='Q', n_jobs=-1) 
Y_hat_df = fcst.forecast(h=4)

# Reconcile the base predictions
Y_train_df = Y_train_df.reset_index().set_index('unique_id')
Y_hat_df = Y_hat_df.reset_index().set_index('unique_id')
reconcilers = [BottomUp(),
               MinTrace(method='ols')]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df,
                          S=S, tags=tags)
Y_rec_df.groupby('unique_id').head(2)